In [5]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from utils import PkdbModel, codeine_idx, pktype_data, filter_out, convert_unit, unstring
plt.style.use('seaborn-whitegrid')
import numpy as np

In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
all_complete = PkdbModel("all_complete", destination="2-merged")
all_complete.read()
all_complete.data.loc[(all_complete.data[("sex","choice")] == "M"),("oral contraceptives","choice")] = "N"
complete_codeine = all_complete.data[codeine_idx(all_complete.data)]

/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2903: DtypeWarning: Columns (21,22,23,33,47,59,60,77,79,96,98,100,103,107,111,112,122,130,131,132,142,143,151,152,154,157,166,168,172,174,175,176,179,187,194,195,202,207,212,217,222,230,242,243,250,266) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [8]:
codeine_clearance = pktype_data(complete_codeine,"clearance")
#codeine_clearance = filter_out(caffeine_clearance,"unit",["ml/min/1.73m^2"])
#caffeine_clearance = filter_out(caffeine_clearance,"study",["Balogh1992"])

codeine_clearance[("weight","value")] = pd.to_numeric(codeine_clearance[("weight","value")])
codeine_clearance[("weight","mean")] = pd.to_numeric(codeine_clearance[("weight","mean")])
codeine_clearance["value_intervention"] = pd.to_numeric(codeine_clearance["value_intervention"])
codeine_clearance["median"] = pd.to_numeric(codeine_clearance["median"])
codeine_clearance["mean"] = pd.to_numeric(codeine_clearance["mean"])
codeine_clearance["value"] = pd.to_numeric(codeine_clearance["value"])
codeine_clearance["se"] = pd.to_numeric(codeine_clearance["se"])
codeine_clearance["sd"] = pd.to_numeric(codeine_clearance["sd"])
codeine_clearance["cv"] = pd.to_numeric(codeine_clearance["cv"])

/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/janekg89/Envs/pkdb_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

In [10]:
codeine_clearance_rel = convert_unit(codeine_clearance,
                             unit_in="mg",
                             unit_out="mg/kg", 
                             factor=1.0/codeine_clearance[("weight","value")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'], 
                             subset=[("weight","value"),"value"])

codeine_clearance_abs = convert_unit(codeine_clearance,
                             unit_in="mg/kg",
                             unit_out="mg", 
                             factor=codeine_clearance[("weight","value")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'],
                             subset=[("weight","value"),"value"])

codeine_clearance = pd.concat([codeine_clearance,codeine_clearance_rel,codeine_clearance_abs], ignore_index=True)

codeine_clearance_rel = convert_unit(codeine_clearance,
                             unit_in="l/h",
                             unit_out="l/h/kg", 
                             factor=1.0/codeine_clearance[("weight","value")],
                             unit_field="unit", 
                             data_fields=['value'], 
                             subset=[("weight","value"),"value"])

codeine_clearance_abs_i = convert_unit(codeine_clearance,
                             unit_in="l/h/kg",
                             unit_out="l/h", 
                             factor=codeine_clearance[("weight","value")],
                             unit_field="unit", 
                             data_fields=['value'],
                             subset=[("weight","value"),"value"])

codeine_clearance = pd.concat([codeine_clearance,codeine_clearance_rel,codeine_clearance_abs_i], ignore_index=True)

codeine_clearance_rel = convert_unit(codeine_clearance,
                             unit_in="mg",
                             unit_out="mg/kg", 
                             factor=1.0/codeine_clearance[("weight","mean")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'], 
                             subset=[("weight","mean"),"mean"])

codeine_clearance_abs = convert_unit(codeine_clearance,
                             unit_in="mg/kg",
                             unit_out="mg", 
                             factor=codeine_clearance[("weight","mean")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'],
                             subset=[("weight","mean"),"mean"])
codeine_clearance = pd.concat([codeine_clearance,codeine_clearance_rel,codeine_clearance_abs], ignore_index=True)


codeine_clearance_rel = convert_unit(codeine_clearance,
                             unit_in="l/h",
                             unit_out="l/h/kg", 
                             factor=1.0/codeine_clearance[("weight","mean")],
                             unit_field="unit", 
                             data_fields=['mean','median','sd', 'se', "cv"],
                             subset=[("weight","mean"),"mean"])

codeine_clearance_abs = convert_unit(codeine_clearance,
                             unit_in="l/h/kg",
                             unit_out="l/h", 
                             factor=codeine_clearance[("weight","mean")],
                             unit_field="unit", 
                             data_fields=['mean','median','sd', 'se', "cv"],
                             subset=[("weight","mean"),"mean"])


codeine_clearance = pd.concat([codeine_clearance,codeine_clearance_rel,codeine_clearance_abs], ignore_index=True)

In [14]:
codeine_clr = PkdbModel("codeine_clearance",destination="3-inferred")
codeine_clr.data = codeine_clearance
codeine_clr.save()
codeine_clearance.groupby(["unit","unit_intervention"]).count()

study  output_type  pk  tissue  substance_name  \
unit   unit_intervention                                                   
l/h    mg                    25           25  25      25              25   
       mg/kg                 18           18  18      18              18   
l/h/kg mg                    18           18  18      18              18   
       mg/kg                 18           18  18      18              18   

                          pktype  interventions  individual_pk  value  \
unit   unit_intervention                                                
l/h    mg                     25             25             16     16   
       mg/kg                  18             18             16     16   
l/h/kg mg                     18             18             16     16   
       mg/kg                  18             18             16     16   

                          group_pk         ...          \
unit   unit_intervention                   ...           
l/h    mg                        9         ...           
       mg/kg                     2         ...           
l/h/kg mg                        2         ...           
       mg/kg                     2         ...           

                          (metabolic ratio, count)  (metabolic ratio, max)  \
unit   unit_intervention                                                     
l/h    mg                                        0                       0   
       mg/kg                                     0                       0   
l/h/kg mg                                        0                       0   
       mg/kg                                     0                       0   

                          (metabolic ratio, mean)  (metabolic ratio, min)  \
unit   unit_intervention                                                    
l/h    mg                                       0                       0   
       mg/kg                                    0                       0   
l/h/kg mg                                       0                       0   
       mg/kg                                    0                       0   

                          (metabolic ratio, unit)  (fat weight, count)  \
unit   unit_intervention                                                 
l/h    mg                                       0                    0   
       mg/kg                                    0                    0   
l/h/kg mg                                       0                    0   
       mg/kg                                    0                    0   

                          (fat weight, cv)  (fat weight, mean)  \
unit   unit_intervention                                         
l/h    mg                                0                   0   
       mg/kg                             0                   0   
l/h/kg mg                                0                   0   
       mg/kg                             0                   0   

                          (fat weight, se)  (fat weight, unit)  
unit   unit_intervention                                        
l/h    mg                                0                   0  
       mg/kg                             0                   0  
l/h/kg mg                                0                   0  
       mg/kg                             0                   0  

[4 rows x 264 columns]

## Timecourses

In [17]:
codeine_timecourse = pktype_data(complete_codeine,"concentration")
codeine_timecourse = filter_out(codeine_timecourse,"output_type",["outputs"])
codeine_timecourse = filter_out(codeine_timecourse,"unit",["nmol/l"])
codeine_timecourse = filter_out(codeine_timecourse,"unit",["nmol/ml"])

codeine_timecourse.groupby(["unit","unit_intervention"]).count()

study  output_type  pk  tissue  substance_name  \
unit  unit_intervention                                                   
ng/g  mg                     1            1   1       1               1   
µg/ml mg                    15           15  15      15              15   
      mg/kg                  2            2   2       2               2   

                         pktype  interventions  individual_pk  value  \
unit  unit_intervention                                                
ng/g  mg                      1              1              0      0   
µg/ml mg                     15             15              2      2   
      mg/kg                   2              2              0      0   

                         group_pk         ...          \
unit  unit_intervention                   ...           
ng/g  mg                        1         ...           
µg/ml mg                       13         ...           
      mg/kg                     2         ...           

                         (metabolic ratio, count)  (metabolic ratio, max)  \
unit  unit_intervention                                                     
ng/g  mg                                        0                       0   
µg/ml mg                                        0                       0   
      mg/kg                                     0                       0   

                         (metabolic ratio, mean)  (metabolic ratio, min)  \
unit  unit_intervention                                                    
ng/g  mg                                       0                       0   
µg/ml mg                                       0                       0   
      mg/kg                                    0                       0   

                         (metabolic ratio, unit)  (fat weight, count)  \
unit  unit_intervention                                                 
ng/g  mg                                       0                    0   
µg/ml mg                                       0                    0   
      mg/kg                                    0                    0   

                         (fat weight, cv)  (fat weight, mean)  \
unit  unit_intervention                                         
ng/g  mg                                0                   0   
µg/ml mg                                0                   0   
      mg/kg                             0                   0   

                         (fat weight, se)  (fat weight, unit)  
unit  unit_intervention                                        
ng/g  mg                                0                   0  
µg/ml mg                                0                   0  
      mg/kg                             0                   0  

[3 rows x 264 columns]

In [18]:
codeine_tc = PkdbModel("codeine_timecourse",destination="3-inferred")
codeine_tc.data = codeine_timecourse
codeine_tc.to_array()
codeine_timecourse = codeine_tc.data

In [19]:
codeine_timecourse_rel = convert_unit(codeine_timecourse,
                             unit_in="mg",
                             unit_out="mg/kg", 
                             factor=1.0/codeine_timecourse[("weight","value")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'], 
                             subset=[("weight","value"),"value"])

codeine_timecourse_abs = convert_unit(codeine_timecourse,
                             unit_in="mg/kg",
                             unit_out="mg", 
                             factor=codeine_timecourse[("weight","value")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'],
                             subset=[("weight","value"),"value"])



codeine_timecourse = pd.concat([codeine_timecourse,codeine_timecourse_rel,codeine_timecourse_abs], ignore_index=True)


codeine_timecourse_rel = convert_unit(codeine_timecourse,
                             unit_in="µg/ml",
                             unit_out="µg/ml/kg", 
                             factor=1.0/codeine_timecourse[("weight","value")],
                             unit_field="unit", 
                             data_fields=['value'], 
                             subset=[("weight","value"),"value"])

codeine_timecourse = pd.concat([codeine_timecourse,codeine_timecourse_rel], ignore_index=True)

codeine_timecourse_rel = convert_unit(codeine_timecourse,
                             unit_in="mg",
                             unit_out="mg/kg", 
                             factor=1.0/codeine_timecourse[("weight","mean")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'], 
                             subset=[("weight","mean"),"mean"])

codeine_timecourse_abs = convert_unit(codeine_timecourse,
                             unit_in="mg/kg",
                             unit_out="mg", 
                             factor=codeine_timecourse[("weight","mean")],
                             unit_field="unit_intervention", 
                             data_fields=['value_intervention'],
                             subset=[("weight","mean"),"mean"])
codeine_timecourse = pd.concat([codeine_timecourse,codeine_timecourse_rel,codeine_timecourse_abs], ignore_index=True)


codeine_timecourse_rel = convert_unit(codeine_timecourse,
                             unit_in="µg/ml",
                             unit_out="µg/ml/kg", 
                             factor=1.0/codeine_timecourse[("weight","mean")],
                             unit_field="unit", 
                             data_fields=['mean','median','sd', 'se', "cv"],
                             subset=[("weight","mean"),"mean"])


codeine_timecourse = pd.concat([codeine_timecourse,codeine_timecourse_rel], ignore_index=True)

In [20]:
codeine_timecourse.groupby(["unit","unit_intervention","time_unit"]).count()

study  output_type  pk  tissue  \
unit     unit_intervention time_unit                                   
ng/g     mg                h              1            1   1       1   
µg/ml    mg                h             17           17  17      17   
         mg/kg             h              6            6   6       6   
µg/ml/kg mg                h              6            6   6       6   
         mg/kg             h              6            6   6       6   

                                      substance_name  pktype  interventions  \
unit     unit_intervention time_unit                                          
ng/g     mg                h                       1       1              1   
µg/ml    mg                h                      17      17             17   
         mg/kg             h                       6       6              6   
µg/ml/kg mg                h                       6       6              6   
         mg/kg             h                       6       6              6   

                                      individual_pk  value  group_pk  \
unit     unit_intervention time_unit                                   
ng/g     mg                h                      0      0         1   
µg/ml    mg                h                      2      2        15   
         mg/kg             h                      2      2         4   
µg/ml/kg mg                h                      2      2         4   
         mg/kg             h                      2      2         4   

                                             ...          \
unit     unit_intervention time_unit         ...           
ng/g     mg                h                 ...           
µg/ml    mg                h                 ...           
         mg/kg             h                 ...           
µg/ml/kg mg                h                 ...           
         mg/kg             h                 ...           

                                      (metabolic ratio, count)  \
unit     unit_intervention time_unit                             
ng/g     mg                h                                 0   
µg/ml    mg                h                                 0   
         mg/kg             h                                 0   
µg/ml/kg mg                h                                 0   
         mg/kg             h                                 0   

                                      (metabolic ratio, max)  \
unit     unit_intervention time_unit                           
ng/g     mg                h                               0   
µg/ml    mg                h                               0   
         mg/kg             h                               0   
µg/ml/kg mg                h                               0   
         mg/kg             h                               0   

                                      (metabolic ratio, mean)  \
unit     unit_intervention time_unit                            
ng/g     mg                h                                0   
µg/ml    mg                h                                0   
         mg/kg             h                                0   
µg/ml/kg mg                h                                0   
         mg/kg             h                                0   

                                      (metabolic ratio, min)  \
unit     unit_intervention time_unit                           
ng/g     mg                h                               0   
µg/ml    mg                h                               0   
         mg/kg             h                               0   
µg/ml/kg mg                h                               0   
         mg/kg             h                               0   

                                      (metabolic ratio, unit)  \
unit     unit_intervention time_unit                            
ng/g     mg                h                                0   
µg/ml    mg                h             

In [21]:
codeine_tc = PkdbModel("codeine_timecourse",destination="3-inferred")
codeine_tc.data = codeine_timecourse
codeine_tc.save()